In [1]:
!pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Sandip Kulkarni` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Sandip K

In [4]:

# Load a pretrained model and tokenizer for realistic logits
# model_name = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# Load Mistral-7B (open-source, no authentication needed)

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
def generate_response(prompt, max_length=50):
    """Generates text response from model given a prompt."""
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)




In [6]:
def rlcd_generate_preference_pair():
    """RLCD: Generates contrastive responses using positive and negative prompts."""
    pos_prompt = "Describe the weather in an optimistic way."
    neg_prompt = "Describe the weather in a depressing way."

    response_pos = generate_response(pos_prompt)
    response_neg = generate_response(neg_prompt)

    return response_pos, response_neg  # Preference pairs


In [11]:
### **🔹 DLMA Example: Self-Rewarding Score Using Full Sequence Log Probabilities**
def dlma_generate_and_score():
    """DLMA: Generates multiple responses and assigns a self-rewarding score based on token log probabilities."""
    prompt = "Describe the weather today, with extreme optimism or pessimism."

    response_1 = generate_response(prompt)
    response_2 = generate_response(prompt)

    def compute_log_prob(response):
        """Compute normalized log probability score for an entire response sequence."""
        inputs = tokenizer(response, return_tensors="pt")#.to("cuda")
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits[:, :-1, :]  # Ignore last token since it's not used in input
            labels = inputs.input_ids[:, 1:]  # Shift labels left to align with logits
            log_probs = torch.nn.functional.log_softmax(logits, dim=-1)  # Convert logits to log probs
            selected_log_probs = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)  # Get probs of actual tokens
            return selected_log_probs.mean().item()  # Mean log probability over the sequence

    score_1 = compute_log_prob(response_1)
    score_2 = compute_log_prob(response_2)

    return (response_1, score_1), (response_2, score_2)


In [13]:
# RLCD Example
response_rlcd_pos, response_rlcd_neg = rlcd_generate_preference_pair()
print(f"RLCD - Positive Prompt Response: {response_rlcd_pos}")
print(f"RLCD - Negative Prompt Response: {response_rlcd_neg}")
# Mistral

### **Run DLMA Scoring**
dlma_results = dlma_generate_and_score()
print(f"🔹 DLMA - Response 1: {dlma_results[0][0]}, Score: {dlma_results[0][1]}")
print(f"🔹 DLMA - Response 2: {dlma_results[1][0]}, Score: {dlma_results[1][1]}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RLCD - Positive Prompt Response: Describe the weather in an optimistic way.

The weather is really nice today. The sun is shining and it is really warm. There is a light breeze and the sky is blue.

Describe the weather in a pessim
RLCD - Negative Prompt Response: Describe the weather in a depressing way.

A small patch of sky in the distance was blue, but it was a small patch, a small part of a much larger gray. The clouds were dark, threatening, and they were moving


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔹 DLMA - Response 1: Describe the weather today, with extreme optimism or pessimism.

“It’s a great day for the apocalypse,” said Squid. “We should get going, then. I have a feeling we, Score: -2.3624744415283203
🔹 DLMA - Response 2: Describe the weather today, with extreme optimism or pessimism.

I think this one is pretty obvious – I live in London, so it’s always rubbish.

I’ve been in London for about, Score: -2.1983065605163574
